In [ ]:
from dataclasses import dataclass
from z3 import *
@dataclass
class EGraph():
    E: list[tuple(ExprRef, ExprRef)]
    T: dict[SortRef]
    def __init__(self):
        self.E = []
        self.T = defaultdict(set)
        self.solver = Solver()
    def add_term(self, t):
        self.T[t.sort()].add(t)
        if is_app(t):
            for t in t.children():
                self.add_term(t)
    def union(self, t1, t2, reason=None):
        # use solver for subsumption check rather than reducing.
        self.solver.push()
        self.solver.add(t1 != t2)
        res = self.solver.check()
        self.solver.pop()
        if res == sat: # non redundant info
            self.solver.assert_and_track(t1 == t2, ) # add and track?
            self.E.append((t1,t2, reason)) # add reason
    def rw(self, sorts, f):
        for xs in product(*[self.T[sort] for sort in sort]):
            lhs, rhs = f(xs)
            self.solver.push()
            # this avoids the reduction step to check if in self.T
            self.solver.add(Not(Or(lhs == t for t in self.T[lhs.sort()]))
            res = self.solver.check() # can give unsat core reason for why lhs == pat[xs]
            self.solver.pop()
            if res == unsat:
                core = self.get_unsat_core()

                self.union(lhs, rhs, reason=("rule", core))




Can we do a contextual egraph using z3 SAT?
Twee style context

"in" the T set can be done as an smt call

solver.push()
s.add(Not(Or(pat == t for t in T)))
res = s.check()
s.pop()
if unsat: # match
    s.unsat_core() #? useful for proof?


# Confluence Competition

Moca conlfuecne compeitition.
logically constrained rewrite systems. THis is similar to egraphs modulo theories? https://project-coco.uibk.ac.at/2024/categories/lctrs.php 
The ARI format. I like this https://project-coco.uibk.ac.at/ARI/index.php
https://ari-cops.uibk.ac.at/ARI/
